In [7]:
pip install pyodbc

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pyodbc


In [8]:
server = 'tcp:mycmvserver.database.windows.net' 
database = 'cmvdb_uat' 
username = 'ecervantes@cmvlatam.com' 
password = 'Columbia87/' 


connection = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=tcp:mycmvserver.database.windows.net,1433;Database=cmvdb_uat;Uid='+username+';Pwd='+password+';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;Authentication=ActiveDirectoryInteractive')
cursor = connection.cursor()

In [9]:
# Define the database URL
db_url = "mssql+pyodbc://ecervantes:artline87/@mycmvserver.database.windows.net.cmvdb_uat"
# Create the engine object
engine = create_engine(db_url)



In [15]:
query = "SELECT TOP (20) * FROM [dbo].[fxblotter.t]"

# Read the SQL query into a DataFrame
fxblotter = pd.read_sql(query, connection)

# Show the DataFrame's head
fxblotter.set_index('tradeid')

,tiquetid,createdat,accionid,sideid,clienteid,comccyid,montocom,px_sp,venccyid,montoven,estadoid,valuedate,monex_pxlast,monex_pp,ejecutivoid,dealerid,px_cover,productotipoid,solicitud
tradeid,,,,,,,,,,,,,,,,,,,
1.0,1,2021-05-26 22:07:00.000,1,1,22,1,600.0,610.0,1,615.0,1,2021-03-16,618.0,617.00,2,1.0,800.0,2,Hola!!! Esto es una prueba desde Teams
1.0,2,2021-05-18 04:19:00.000,1,2,2,1,800.0,710.0,2,900.0,3,2021-05-18,620.0,619.00,3,1.0,610.0,2,Vamoneeee
NaN,5,NaT,1,2,2,1,800.0,710.0,2,900.0,3,None,NaN,NaN,3,NaN,610.0,2,Vamoneeee
NaN,6,2021-05-26 22:07:00.000,1,1,22,1,600.0,610.0,2,615.0,1,None,NaN,NaN,2,NaN,800.0,1,Hola!!! Esto es una prueba 12345
NaN,7,NaT,3,2,6,2,96897.0,123124.0,1,12345.0,3,None,NaN,NaN,1,NaN,12344.0,2,xxxxx
NaN,8,NaT,3,2,33,1,11234.0,141234.0,1,1234.0,5,None,NaN,NaN,2,NaN,2134.0,2,1431241
NaN,9,NaT,2,1,3,1,900.0,1000.0,1,900.0,1,None,NaN,NaN,2,NaN,1000.0,2,prueba en vivo
NaN,10,NaT,1,1,0,1,1000.0,800.0,1,1000.0,1,None,NaN,NaN,2,NaN,444.0,1,prueba con el combo box
NaN,11,NaT,1,1,116244,2,NaN,615.0,1,1000.0,2,None,NaN,NaN,1,NaN,616.0,1,None


In [10]:
fxblotter[fxblotter['ccycom']=='USD']['montoven'].sum()


4215000.0

In [6]:
monexpp = 610.00
monexbid = 0
monexbidpp = 0
monexask = 0
monexaskpp = 0

###FX Posicion Import fxblotter.t as a DF to make calculations###
bidtotal = fxblotter[fxblotter['ccycom']=='USD']['montocom'].sum()
asktotal = fxblotter[fxblotter['ccyven']=='USD']['montoven'].sum()
fxnetpos = bidtotal - asktotal ### USD Moneda base ###
## Weighted average 
bidpp = fxblotter[fxblotter['ccycom']=='USD']['montoven'].sum() / bidtotal
askpp = fxblotter[fxblotter['ccyven']=='USD']['montocom'].sum() / asktotal
##PnL##
coveredpnl =  min(bidtotal, asktotal) * (askpp - bidpp)  
###uncoveredpnl
if fxnetpos > 0:
    uncoveredpnl = (bidtotal - asktotal) * (monexpp - bidpp)
else :   
    uncoveredpnl = (asktotal - bidtotal) * (askpp - monexpp)
totalpnl = coveredpnl + uncoveredpnl


print(f"Bid weighted average price:  {bidpp}") 
print(f"Bid total amount:  {bidtotal}")
print(f"Ask weighted average price:  {askpp}") 
print(f"Ask total amount:  {asktotal}")
print(f"FX net position:  {fxnetpos}")
print(f"daily total pnl is: {totalpnl}")

Bid weighted average price:  603.0
Bid total amount:  5000.0
Ask weighted average price:  620.0
Ask total amount:  3000.0
FX net position:  2000.0
daily total pnl is: 65000.0


In [7]:
##Deal Management Blotter###
qtrades = fxblotter['side'].value_counts()
qestado = fxblotter['estado'].value_counts()
print(qtrades)
print(qestado)


bid           3
ask           1
Name: side, dtype: int64
aplicada    4
Name: estado, dtype: int64


In [ ]:
or

bidpp = fxblotter[fxblotter['ccycom']=='USD']['montoven'].sum() / bidtotal
askpp = 




 # Write the SQL query
query = """
SELECT agent_id, COUNT(region_id) as region_count
FROM agent_region_junction
GROUP BY agent_id
"""

# Read the SQL query into a DataFrame
agent_region_df = pd.read_sql(query, engine)

# Show the DataFrame's head
agent_region_df

## FX Posicion

<mark>Import fxblotter.t as a DF to make calculations</mark>

<mark>netpos = \[monto total de ventas (USD moneda base)\]  -  \[monto total de compra (USD moneda base)\]</mark>

## Stats

asktotal y askpp 

bidtotal y bidpp

## PnL 

PnL =  min \[total offer ,total bid \] \* \[ pp de offer  -  pp de bid\]  + if \[netpos \>0 , (bidtotal - offertotal) \* (monex.pp - bidpp), (offertotal - bidtotal) \* (offerpp - monexpp) \]

## DMB

compras, ventas, cantidad por 'estado', monex compra venta

## <span style="font-size: 14px;">definir compras y ventas de monex como variables float insert</span>

<span style="font-size: 14px;"><br></span>